In [8]:
%%capture
!pip install transformers
!pip install gdown
!pip install opencc-python-reimplemented
!pip install googletrans==3.1.0a0
!pip install fake_useragent

In [9]:
def save():
  %notebook 鉅亨網文字探勘.ipynb 
  !cp 鉅亨網文字探勘.ipynb "/content/drive/MyDrive/Colab Notebooks"

In [10]:
#save()

In [11]:
import torch
import numpy as np
import pandas as pd
import re

In [12]:
from transformers import BertModel, BertTokenizer
PATH = '/content/FinBERT_L-12_H-768_A-12_pytorch'

In [13]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent


In [14]:
import time
import datetime as dt
import time
import tqdm

In [15]:
from opencc import OpenCC
cc = OpenCC('t2s')

In [ ]:
#!gdown https://drive.google.com/uc?id=1qW1YWtw3q9Q28QThrIY-rDU9Gl-SLIKO
#!unzip /content/FinBERT_L-12_H-768_A-12_pytorch.zip

# 財金BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PATH ,local_files_only=True)
model = BertModel.from_pretrained(PATH ,local_files_only=True)

In [ ]:
sector_dict = {
'健護': '醫療保健服務類股',
'科技': '科技類股', 
'公用事業': '公用事業類股',
'生技':'生技類股', 
'能源':'能源類股',
'替代能源':'替代能源類股',
'消費':'消費品類股',
'金融':'金融類股', 
'礦業':'礦業',
'天然資源':'天然資源', 
'貴金屬現貨':'貴金屬',
'農產品':'農產品',
'貴金屬類股':'貴金屬',
}

In [ ]:
mkt_dict= [
'新興市場債',
'高收益債',
'全球債',
'投資等級公司債',
'美國公債',
'亞債',
'REITs不動產投資信託',
'新臺幣類貨幣',
'外幣類貨幣',
'亞太股市', '美國股市', '全球新興市場', 
'全球型股市', '大中華股市', '中國A股', '印度股市','歐洲股市', 
'東南亞股市', '拉丁美洲股市', 
'韓國股市', '新興歐洲股市', '德國股市', '台灣股市','俄羅斯股市',
'中國H股', '日本股市', '英國股市','泰國股市',
'香港股市', '澳洲股市', '印尼股市', '巴西股市', '歐非中東股市'
]


In [ ]:
mkt_dict = {x:x for x in mkt_dict}
mkt_dict['亞債'] = '亞洲債券'
mkt_dict['REITs不動產投資信託'] = '不動產'

In [ ]:
total_sec_dict = {**sector_dict, **mkt_dict}

In [ ]:
sentences = list(set(total_sec_dict.values()))
sentences = [cc.convert(x) for x in sentences]

In [ ]:
inputs = tokenizer(sentences, padding=True, return_tensors="pt")


In [ ]:
## 單純forward pass 沒要計算梯度
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
x = outputs['last_hidden_state']
np.savetxt('bert_mkt_emb.tsv', x[:,0].cpu().detach().numpy(), delimiter="\t")
pd.DataFrame(sentences).to_csv('bert_mkt_emb_meta.tsv', header=False, index=False, sep="\t")

# 鉅亨網爬蟲 初步整理


設定UserAgent

In [18]:
ua = UserAgent()
headers = {'user-agent': ua.random}

In [19]:
headers

{'user-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.0; de-DE) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.3 Safari/533.19.4'}

### 原始方法 
直接從頁面爬取

In [16]:
res = requests.get('https://news.cnyes.com/news/cat/wd_stock')
soup = BeautifulSoup(res.text, 'html.parser')


news_list = soup.find_all("a", attrs={"class":"_1Zdp"})
[x['href'] for x in news_list]
[x.find('h3').get_text(strip=True) for x in news_list

## .get_text(strip=True) 可避免不必用空白符亂碼

## 將資料起訖日轉換成unix timestamp

一種時間格式，從UTC1970年1月1日0時0分0秒起至現在的總秒數


In [21]:
## 將資料起迄日轉換unix timestamp
## 從20210101 - 20211115
start_unix_timestamp = int(time.mktime(dt.date(2021,1,1).timetuple()))
end_unix_timestamp = int(time.mktime(dt.date(2021,11,15).timetuple()))

## 直接爬內部API 
好處多多

格式化字串(format string) 小教學

In [27]:
print('hi, {}'.format('Hello!'))

print('hi, I am {name}'.format(name = 'Alex'))

hi, Hello!
hi, I am Alex


將起訖日參數填入，爬取第一頁
從回傳的資料以此先得知總共有幾頁要爬

In [28]:
url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt={}&endAt={}&limit=30&page={}'.format(
    start_unix_timestamp, 
    end_unix_timestamp, 
    1
)
res = requests.get(url_api, headers=headers)
res_json = res.json()

In [29]:
last_page = res_json['items']['last_page']
last_page

613

開始把613頁全爬完

In [ ]:
news_data_list = []

for i in tqdm.tqdm(range(1,last_page+1)):
  url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt={}&endAt={}&limit=30&page={}'.format(start_unix_timestamp, end_unix_timestamp, i)
  res = requests.get(url_api, headers=headers)
  res_json = res.json()
  news_data_list.append(res_json)
  time.sleep(1)

100%|██████████| 613/613 [23:20<00:00,  2.29s/it]


寫入pickle

In [32]:
import pickle
'''
with open('鉅亨網2021新聞.pickle', 'wb') as f:
  pickle.dump(news_data_list, f)
'''

"\nwith open('鉅亨網2021新聞.pickle', 'wb') as f:\n  pickle.dump(news_data_list, f)\n"

# 資料讀入

In [33]:
!gdown https://drive.google.com/uc?id=1ATL1IdoLgLfV-ro1r5BQzPp8DZXVqfws

Downloading...
From: https://drive.google.com/uc?id=1ATL1IdoLgLfV-ro1r5BQzPp8DZXVqfws
To: /content/___2021__.pickle
100% 61.3M/61.3M [00:00<00:00, 147MB/s]


In [35]:
with open('/content/___2021__.pickle', 'rb') as f:
  news_data_list = pickle.load(f)

正則表達式:  只保留英文中文數字 


In [ ]:
def remove_punctuation(line):
  #rule = re.compile("[^a-z A-Z \u4e00-\u9fa5]")
  rule = re.compile("[^a-z A-Z 0-9 \u4e00-\u9fa5]")
  line = rule.sub('',line)
  return line


正則表達式: 去除亂碼


In [36]:
## 去除亂碼
def remove_char(line):
  rule = re.compile('&lt;.+?(&gt;|>)')
  line = rule.sub('',line)
  return line


UNIX timestamp轉日期

In [ ]:
from datetime import datetime
ts = int('1636969777')

## UNIX timestamp轉日期
print(dt.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2021-11-15 09:49:37


從Json中出取資訊

- content: 本文
- summary: 新聞摘要
- tittle: 新聞標題
- market: 新聞提到的標的


In [37]:
news_data_list[0]['items']['data'][0]

{'abTesting': None,
 'categoryId': 827,
 'categoryName': '台股新聞',
 'columnists': None,
 'content': '&lt;p&gt;混凝土廠國產 (2504-TW) 今 (15) 日公告，將高雄「悅誠廣場」房地持分以總價 23.5 億元，出售予興富發 (2542-TW) 子公司碧江企業，處分利益達 14.9 億元，估貢獻每股純益約 1.27 元，預計第四季至明年第一季財報入帳。&lt;/p&gt;\n\n&lt;p&gt;國產這次售出的「悅誠廣場」土地資產，土地面積 4053 坪，為地上 3 樓、地下 2 樓的建物，總樓地板面積約 1.2 萬坪，由國產與其他地主共同持分，2016 年由興富發向國產等地主承租。&lt;/p&gt;\n\n&lt;p&gt;國產強調，目前全台仍擁有 45.7 萬坪的資產，會配合整體經濟現勢與區域發展趨勢，借力於未來重大交通建設與都市更新相關的商圈規劃，發展出綜合商場、渡假旅館 / 國際精品飯店、多功能辦公大樓、休閒農場等多項主題事業。&lt;/p&gt;\n\n&lt;p&gt;國產董事會日前也通過 158 億餘元的投資預算，因應「南港大型開發案」相關營造發包等作業，該案佔地 6500 坪，預估開發總樓地板面積約 6 萬餘坪，興建完成後，國產可分回 4.6 萬建坪及約 550 個停車位。&lt;/p&gt;\n',
 'coverSrc': {'l': {'height': 359,
   'src': 'https://cimg.cnyes.cool/prod/news/4771793/l/7e7c47f106072253eb6e53271586570d.jpg',
   'width': 640},
  'm': {'height': 213,
   'src': 'https://cimg.cnyes.cool/prod/news/4771793/m/7e7c47f106072253eb6e53271586570d.jpg',
   'width': 380},
  's': {'height': 101,
   'src': 'https://cimg.cnyes.cool/prod/news/4771793/s/7e7c47f106072253

轉換為dataframe

In [38]:
news_extracted_list=[]

for chunk in news_data_list:
  article_content = chunk['items']['data']
  data_dict = [{
      'id':x['newsId'], 
      'title':x['title'], 
      'summary':x['summary'],  
      'article':remove_char(x['content']),  
      #'stock':x['market'],

      'unix_timestamp':x['publishAt'],
      'datetime':dt.datetime.utcfromtimestamp(x['publishAt']).strftime('%Y-%m-%d')
      } for x in article_content
  ]
  news_extracted_list.extend(data_dict)


In [39]:
org_corpus_df = pd.DataFrame(news_extracted_list)

去除換行符號與空白

In [41]:
org_corpus_df['article'] = org_corpus_df['article'].str.replace('\n\n',"")
org_corpus_df['article'] = org_corpus_df['article'].str.replace(" ", "")

In [42]:
org_corpus_df.head()

,id,title,summary,article,unix_timestamp,datetime
0,4771793,國產出售高雄悅誠廣場資產 貢獻EPS 1.27元,混凝土廠國產 (2504-TW) 今 (15) 日公告，將高雄「悅誠廣場」房地持分以總價 2...,混凝土廠國產(2504-TW)今(15)日公告，將高雄「悅誠廣場」房地持分以總價23.5億元...,1636984978,2021-11-15
1,4771790,〈台泥法說〉Q4煤價下滑成本壓力減 毛利率將回升至平均之上,台泥 (1101-TW) 今 (15) 日召開法說，總經理李鐘培表示，10 月以來中國能耗雙...,台泥(1101-TW)今(15)日召開法說，總經理李鐘培表示，10月以來中國能耗雙控持續，水...,1636984833,2021-11-15
2,4771785,〈台泥法說〉張安平談氣候變遷「水泥業站在生命第一現場」 承擔減碳責無旁貸,台泥 (1101-TW) 董事長張安平今 (15) 日在法說會前致詞時指出，今天的氣候災害，...,台泥(1101-TW)董事長張安平今(15)日在法說會前致詞時指出，今天的氣候災害，是人類打...,1636984795,2021-11-15
3,4771771,達邁調整產品組合 看好高頻高速傳輸及汽車市場應用,聚醯亞胺（PI）薄膜廠達邁 (3645-TW) 積極展開產品應用多元化的布局，由以往高達 9...,聚醯亞胺（PI）薄膜廠達邁(3645-TW)積極展開產品多元化布局，以往高達98%的軟板應用...,1636980649,2021-11-15
4,4771770,22家壽險前3季合賺3389億元破紀錄 淨值比all pass！,22 家壽險前 3 季合賺 3389 億元破紀錄 淨值比 all pass！,壽險公司第3季財報全數公布，22家壽險公司合計前3季稅後純益逾3389億元，創史上新高，年增...,1636980334,2021-11-15


# 進行分詞

轉成list格式以利分詞

先取用摘要進行探勘

In [43]:
text = org_corpus_df['summary'].astype(str).tolist()

In [45]:
text[0:5]

['混凝土廠國產 (2504-TW) 今 (15) 日公告，將高雄「悅誠廣場」房地持分以總價 23.5 億元，出售予興富發 (2542-TW) 子公司碧江企業，處分利益達 14.9 億元，估貢獻每股純益約 1.27 元。',
 '台泥 (1101-TW) 今 (15) 日召開法說，總經理李鐘培表示，10 月以來中國能耗雙控持續，水泥產銷量減但價格持穩高檔，且近期煤價大幅回落，有利毛利率回升到過往水準之上，加上廣東韶關廠投產也將挹注營收表現。',
 '台泥 (1101-TW) 董事長張安平今 (15) 日在法說會前致詞時指出，今天的氣候災害，是人類打造繁榮的副作用漸漸浮現、失衡的大自然對人類的反撲；與人類文明與大自然息息相關的水泥業「站在生命的第一現場」。',
 '聚醯亞胺（PI）薄膜廠達邁 (3645-TW) 積極展開產品應用多元化的布局，由以往高達 98% 的軟板應用，在今年前三季的比重已降至 64%，而在非軟板的應用的成長到 36% 則以車用、散熱產品為主。',
 '22 家壽險前 3 季合賺 3389 億元破紀錄 淨值比 all pass！']

### 載入停用詞

載入中文之中「的、了、與其、和」這種意義不大詞彙

In [46]:
%%capture
!wget https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt

In [47]:
## 逐行讀入
with open('/content/cn_stopwords.txt', 'r') as f:
  stop_words = f.read().splitlines()

使用opencc套件進行簡轉繁

In [48]:
from opencc import OpenCC
cc = OpenCC('s2t')

## 轉繁體
stop_words = [cc.convert(text) for text in stop_words]

In [49]:
## 保留數字
stop_words = list(filter(lambda x: x not in ['0','1','2','3','4','5','6','7','8','9'], stop_words))

In [52]:
stop_words[100:110]

['介於', '仍', '仍舊', '從', '從此', '從而', '他', '他人', '他們', '以']

### 中研院開發BERT版NLP套件

有分詞、實體辨識、詞性判斷等功能，效果提升但比較慢

In [53]:
%%capture
!pip install -U ckip-transformers
#https://github.com/ckiplab/ckip-transformers


In [54]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

- `level` 1-3: 3準確度高速度最慢
- `device=0`: 採用GPU

In [55]:
## 設定driver
ws_driver = CkipWordSegmenter(level=3,device=0)

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/388M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

開始進行分詞

In [56]:
%%time
corpus_ws_org  = ws_driver(text)

Inference: 100%|██████████| 66/66 [04:34<00:00,  4.16s/it]


CPU times: user 4min 41s, sys: 784 ms, total: 4min 42s
Wall time: 4min 42s


In [57]:
corpus_ws_org[0:5]

[['混凝土廠',
  '國產',
  ' ',
  '(',
  '2504-TW',
  ')',
  ' ',
  '今',
  ' ',
  '(15) ',
  '日',
  '公告',
  '，',
  '將',
  '高雄',
  '「',
  '悅誠',
  '廣場',
  '」',
  '房地',
  '持分',
  '以',
  '總價',
  ' 23.5 億',
  '元',
  '，',
  '出售予',
  '興富發',
  ' ',
  '(',
  '2542-TW',
  ')',
  ' ',
  '子公司',
  '碧江',
  '企業',
  '，',
  '處分',
  '利益',
  '達',
  ' 14.9 億',
  '元',
  '，',
  '估',
  '貢獻',
  '每',
  '股',
  '純益',
  '約',
  ' 1.27',
  ' ',
  '元',
  '。'],
 ['台泥',
  ' ',
  '(',
  '1101-TW',
  ') ',
  '今',
  ' ',
  '(15) ',
  '日',
  '召開',
  '法說',
  '，',
  '總經理',
  '李鐘培',
  '表示',
  '，',
  '10 ',
  '月',
  '以來',
  '中國',
  '能',
  '耗',
  '雙',
  '控',
  '持續',
  '，',
  '水泥',
  '產銷量',
  '減',
  '但',
  '價格',
  '持穩',
  '高檔',
  '，',
  '且',
  '近期',
  '煤價',
  '大幅',
  '回落',
  '，',
  '有利',
  '毛利率',
  '回升',
  '到',
  '過往',
  '水準',
  '之上',
  '，',
  '加上',
  '廣東',
  '韶關廠',
  '投產',
  '也',
  '將',
  '挹注',
  '營收',
  '表現',
  '。'],
 ['台泥',
  ' ',
  '(',
  '1101-TW',
  ')',
  ' ',
  '董事長',
  '張安平',
  '今',
  ' ',
  '(15) ',
  '日',
  '在',
  '法說會',
  

### 中研院Bi-LSTM斷詞

先前推出的，速度較快

In [ ]:
%%capture
!pip install -U ckiptagger[gdown]

In [ ]:

from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
data_utils.download_data_url("./")
ws = WS("./data", disable_cuda=False)

In [ ]:
%%time
corpus_ws_org  = ws(text)

CPU times: user 1min 3s, sys: 7.24 s, total: 1min 11s
Wall time: 1min 11s


## 將處理完的分詞進一步進行清理（停用詞、符號等）

詞性判斷
股票dict

最後在把詞彙用空白join以利套件使用

In [ ]:
## 刪除標點
corpus_ws = [list(map(lambda x: remove_punctuation(x), corpus)) for corpus in corpus_ws_org]

In [ ]:
## 刪除空白的element
corpus_ws = [list(filter(lambda x: x!="" and x!=" ", corpus)) for corpus in corpus_ws]

In [ ]:
## 刪除element裡的空白
pattern = re.compile(r'\s+')
corpus_ws = [list(map(lambda x: re.sub(pattern, '', x), corpus)) for corpus in corpus_ws]

In [ ]:
## 把停用詞刪除
corpus_ws = [[x for x in sentence if x not in stop_words] for sentence in corpus_ws]

In [ ]:
corpus_ws[0]

['混凝土廠',
 '國產',
 '2504TW',
 '15',
 '日',
 '公告',
 '高雄',
 '悅誠',
 '廣場',
 '房地',
 '持分',
 '總價',
 '235',
 '億',
 '元',
 '出售',
 '予',
 '興富',
 '發',
 '2542',
 'TW',
 '子公司',
 '碧江',
 '企業',
 '處分',
 '利益',
 '達',
 '149',
 '億',
 '元',
 '估',
 '貢獻',
 '股',
 '純益',
 '約',
 '127',
 '元']

In [ ]:
org_corpus_df['summary_seg'] = corpus_ws

In [ ]:
org_corpus_df['summart_list_len'] = org_corpus_df['summary_seg'].apply(lambda x: len(x))

In [ ]:
org_corpus_df['summary_seg_join']  = org_corpus_df['summary_seg'].apply(lambda x: " ".join(x))

In [ ]:
org_corpus_df['summary_seg_join'] 

0        混凝土廠 國產 2504TW 15 日 公告 高雄 悅誠 廣場 房地 持分 總價 235 億...
1        台泥 1101TW 15 日 召開 法 說 總經理 李鐘培 表示 10 月 中國 耗 雙控 ...
2        台泥 1101TW 董事長 張安平 15 日 法說會 前 致詞 時 指出 今天 氣候 災害 ...
3        聚醯亞胺 PI 薄膜廠 達邁 3645 TW 積極 展開 產品 應用 多元化 布局 以往 高...
4                  22 家 壽險 前 3季 合賺 3389 億 元 破紀錄 淨值 allpass
                               ...                        
18364    泛國 巨 2327TW 集團 電感廠 奇力 新 2456TW 受惠 5G 手機 伺服器 基地...
18365    環宇 KY 4991TW 去年 受 中國 基礎 建設 招標 遞延 影響 加上 轉投資 晶成 ...
18366    閎暉 3311 TW 近年 積極 調整 體質 過去 手機 按鍵 業務 比例 降低 聚焦 車用...
18367                            潛力股 搭 電動車 成長 順風車 獲利 動能 加溫
18368    全球 車市 需求 可望 新冠肺炎 疫苗 研發 有成 帶動 進一步 扭轉 汽車板廠 過去 淡季...
Name: summary_seg_join, Length: 18369, dtype: object

# Text mining

## 關鍵字萃取

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

### 中文TF-IDF

`token_pattern='\\b\\w+\\b'` : 讓TF-IDF認得單字詞

`lowercase=False`

In [ ]:
!pip3 install jieba

In [ ]:
import jieba
import jieba.analyse

In [ ]:
tags = jieba.analyse.extract_tags(corpus_white_join, topK=10)

AttributeError: ignored

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
text_cv = CountVectorizer(max_df=0.7, min_df=5)

td_matrix = text_cv.fit_transform(corpus_white_join)

In [ ]:
tfidf = TfidfTransformer()

tfidf_matrix = tfidf.fit_transform(td_matrix)

In [ ]:
topk_arr = np.argsort(-tfidf_matrix.toarray(), axis=1)[:,0:5]

In [ ]:
indices = np.nonzero(-tfidf_matrix.toarray())[0]
#indices[np.argsort(-tfidf_matrix.toarray()[indices])][:,0:5]

In [ ]:
corpus_white_join[20:30]

['統包 工程 承攬商 中鼎 9933TW  集團 攜手 欣陸 3703TW 旗下 欣達 環工 執行 桃園市 中壢 地區 污水 下水道 系統 建設 BOT 計畫 今 15 日 就 水 資源 回收 中心 舉行 開工 動土',
 '封測 龍頭 日月光 投控 3711TW 今 15 日 宣布 連續 第六 年 整體 成績 獲 道瓊 永續 指數 半導體 及 半導體 設備 產業 產業 最 高分 為 台灣 首 家 連續 六 度 獲得 產業 最 高分 的 公司',
 '漢磊 3707TW 今 15 日 表示 SiC 產線 第四 季 全數 滿載 整體 毛利率 將 持續 向 上 進一步 提升 至 1518 看好 明年 仍 產能 供不應求 全 年 營收 成長 2 成 起跳 毛利率 也 將 續揚',
 '國產署 近期 與 高雄 屏東 兩 縣市 地方 政府 合作 辦理 2 件 國有 土地 開發案 預計 可 吸引 民間 投資 60億 元 創造 415億 元 產值 與 1422 個 就業 機會',
 '國銀 人民幣 存款 連 3 減 個人戶 連 8 個 月 下滑',
 'PMOLED 廠 錸寶 8104TW 今 15 日 召開 法說 展望 第四 季 錸寶 表示 PMOLED 業務 受 整體 供應鏈 缺料 歐美 塞港 影響 客戶 下單 轉趨 保守 估 本 季 相關 業務 將 面臨 修正 電池 模組 業務 則 在 上游 材料 提前 備貨 下 可望 保持 成長',
 '聯強 2347TW 今 15 日 舉行 法說 副總裁 杜書全 表示 雖然 資訊 商用 成長 力道 放緩 不過 資訊 商用 與 ic 元件 業務 則 受惠 企業 數位 轉型 資料 中心 伺服器 缺料 舒緩 業績 將 明顯 成長 法人 看好 本業 每 股 純益 衝破 6 元',
 '偉詮電 2436TW 今 15 日 公布 10 月 自結 稅 後 純益 053億 元 年增 179 每 股 稅 後 純益 03 元 累計 前 10 月 稅 後 純益 62億 元 年增 204 每 股 稅 後 純益 35 元 受惠 USBPD 出貨 續 旺 加上 10 月 開始 漲',
 '嘉晶 3016TW 今 15 日 表示 部分 客戶 訂單 已 看 至 明年 底 明年 將 持續 調漲 價格 對 展望 樂觀 看待 營收 將 有 雙位數 成長 毛利 也 將 續 

In [ ]:
text_cv.get_feature_names_out()[topk_arr][20:30]

array([['承攬商', 'bot', '3703tw', '9933tw', '欣陸'],
       ['產業', '連續', '半導體', '第六', '3711tw'],
       ['毛利率', '續揚', '3707tw', '漢磊', 'sic'],
       ['國有', '屏東', '國產署', '60億', '民間'],
       ['存款', '國銀', '人民幣', '下滑', '001'],
       ['錸寶', 'pmoled', '業務', '8104tw', '轉趨'],
       ['商用', '資訊', '副總裁', '聯強', '2347tw'],
       ['純益', '10', 'usbpd', '62億', '2436tw'],
       ['嘉晶', '明年', '化合物', '毛利', '元宇宙'],
       ['國賓', '飯店', '簽約', '合建', '改建']], dtype=object)

In [ ]:

np.array(text_cv.get_feature_names_out())[importance][0:100]

NameError: ignored

In [ ]:
text_cv.get_feature_names_out()

array(['001', '003', '004', ..., '龍頭廠', '龐大', '龔明鑫'], dtype=object)

In [ ]:
from gensim.summarization import keywords

In [ ]:
keywords

In [ ]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

In [ ]:
test=".".join(org_corpus_df[org_corpus_df['summart_list_len']!=0]['summary_seg_join'].tolist()[0:10])

In [ ]:
keywords(test , ratio=0.5)

'董事長\n息息相關\n總經理 李鐘培\n化合物 半導體'

## 主題模型

## Word Embedding

## BERT